## 4. Client Test

**NOTE**: 

The client test resides in the `client` directory and mounted to the devcontainer.
That means you can either run this test either server-side or client-side.
There are 3 different ways to run this test:

a) **If you are running from gai-llm-svr devcontainer**:  

    - Press **F5** to start the chat server.    
    - Set base url to **http://localhost:12031** in client_config  
    NOTE: You cannot run the server using `docker-compose up` command in this case, as it will conflict with the devcontainer's port mapping.

b) **If you are running from gai-llm-svr Client**:  

    - Use `docker-compose up` to start the chat server.  
    - Set base url to **http://localhost:12031** in client_config  

c) **If you are running from gai-sdk devcontainer**:  

    - From gai-sdk, open devcontainer.  
    - Set base url to **http://gai-llm-svr:12031** in client_config  

### b) Load the client

In [1]:
import os
os.environ["LOG_LEVEL"] = "INFO"
from gai.llm.client import ChatClient
chat_client = ChatClient({
        "client_type": "gai",
        #"url": "http://localhost:12031/gen/v1/chat/completions"
        "url": "http://gai-llm-svr:12031/gen/v1/chat/completions"
        })

from gai.llm.client import AsyncChatClient
async_client = AsyncChatClient({
        "client_type": "gai",
        #"url": "http://localhost:12031/gen/v1/chat/completions"
        "url": "http://gai-llm-svr:12031/gen/v1/chat/completions"
        })


### c) Stream

In [2]:
result = chat_client.chat(model="ttt",messages="User:Tell me a one paragraph story",stream=True)
for chunk in result:
    print(chunk.extract(),end="",flush=True)

Once upon a time, in a small village nestled between two great mountains, there lived a young girl named Elara. She was known throughout the village for her kindness and intelligence, and everyone loved her dearly. One day, while walking through the forest, Elara stumbled upon a wounded fawn. The fawn was badly injured and could not walk, so Elara took it upon herself to nurse it back to health. She spent day and night by its side, feeding it, cleaning its wounds, and comforting it when it cried. As the days passed, the fawn slowly began to heal, and soon it was frolicking about in the meadow with newfound strength. Elara was overjoyed to see the fawn recover, and she knew that she had made a friend for life. From that day on, Elara and the fawn were inseparable, and the whole village rejoiced in their friendship.{'type': 'finish_reason', 'finish_reason': 'stop'}

In [3]:
result = await async_client.chat(model="ttt",messages="User:Tell me a one paragraph story",stream=True)
async for chunk in result:
    print(chunk.extract(),end="",flush=True)

Once upon a time, in a small village nestled between two great mountains, there lived a wise old man named Thomas. Thomas was known throughout the village for his incredible knowledge of herbs and plants. He spent his days wandering the forests gathering whatever he could find, then returned to his hut to create potions and remedies from them. One day, a young girl named Lily fell ill with a fever that the village doctor couldn't cure. The villagers were desperate for a solution, so they turned to Thomas. He looked at Lily and knew exactly what to do. He mixed up a potion from the rarest of rare herbs that only grew on the mountain tops. The villagers carried Lily up the mountain, and Thomas handed her the potion. She drank it, and within hours, she was back on her feet. From that day on, Thomas was known as the village's greatest healer.{'type': 'finish_reason', 'finish_reason': 'stop'}

### d) Generate

In [4]:
result = chat_client.chat(model="ttt",messages="User:Tell me a one paragraph story",stream=False)
print(result.extract()["content"])

Once upon a time, in a small village nestled between two great mountains, there lived a young girl named Lily. She was known for her kindness and gentle spirit, often helping her neighbors with their everyday tasks. One day, as she was on her way to fetch water from the nearby stream, she stumbled upon a beautiful butterfly with wings of shimmering silver and gold. It lay struggling on the ground, unable to fly. With compassion in her heart, Lily picked it up carefully and decided to take it home, nursing it back to health with her own special care. As the days passed, the butterfly slowly regained its strength and one morning, it finally felt strong enough to fly away. Lily watched happily as it soared into the sky, spreading its majestic wings. From that day on, whenever she saw a butterfly with wings of silver and gold, she would remember the incredible journey of friendship and healing that began with one small act of kindness.


In [4]:
result = await async_client.chat(model="ttt",messages="User:Tell me a one paragraph story",stream=False)
print(result.extract()["content"])

Once upon a time, in a small village nestled between two great mountains, there lived a young girl named Akira. She was known throughout the village for her kindness and generosity, always helping those in need without expecting anything in return. One day, a terrible storm swept through the village, destroying homes and uprooting trees. The villagers were in shock, trying to figure out how to rebuild their lives. Akira knew what she had to do; she gathered her friends and together they started working on rebuilding the houses of their neighbors. They worked tirelessly, day and night, until every house was restored. The villagers were overwhelmed with gratitude for Akira and her friends, and soon the village was thriving once again.


### e) Tool Call

In [4]:
result = chat_client.chat(
        model="ttt",
        messages=[
            {"role":"user","content":"What is the current time in Singapore?"},
            {"role":"assistant","content":""}
        ],
        tool_choice="required",
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "google",
                    "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "search_query": {
                                "type": "string",
                                "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                            }
                        },
                        "required": ["search_query"]
                    }
                }
            }
        ],
        stream=False)
print(result.extract())

{'type': 'function', 'name': 'google', 'arguments': '{"search_query": "current time in Singapore"}'}


In [5]:
result = await async_client.chat(
        model="ttt",
        messages=[
            {"role":"user","content":"What is the current time in Singapore?"},
            {"role":"assistant","content":""}
        ],
        tool_choice="required",
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "google",
                    "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "search_query": {
                                "type": "string",
                                "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                            }
                        },
                        "required": ["search_query"]
                    }
                }
            }
        ],
        stream=False)
print(result.extract())

{'type': 'function', 'name': 'google', 'arguments': '{"search_query": "current time in Singapore"}'}


### f) Structured Output

In [5]:
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

text = """Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.
"""

result = chat_client.chat(
    model="ttt",
    messages=[{'role':'user','content':text},{'role':'assistant','content':''}], 
    json_schema=Book.schema(),
    stream=False)
print(result.extract()["content"])

/tmp/ipykernel_96508/2285534030.py:20: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json_schema=Book.schema(),


{
  "title": "Foundation",
  "summary": "Foundation is a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire.",
  "author": "Isaac Asimov",
  "published_year": 1951
}


In [7]:
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

text = """Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.
"""

result = await async_client.chat(
    model="ttt",
    messages=[{'role':'user','content':text},{'role':'assistant','content':''}], 
    json_schema=Book.schema(),
    stream=False)
print(result.extract()["content"])

/tmp/ipykernel_7977/1987867710.py:20: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json_schema=Book.schema(),


{
  "title": "Foundation",
  "summary": "Foundation is a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire.",
  "author": "Isaac Asimov",
  "published_year": 1951
}
